In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

GPU = True
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-07-27 09:46:01.239777: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-07-27 09:46:14.581579: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-07-27 09:46:14.604098: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-07-27 09:46:14.604131: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-07-27 09:46:14.606447: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-07-27 09:46:14.608660: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10

# Load data from the output of Brusselator model

In [2]:
data_path = os.path.abspath("../turing_Brusselator.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("../turing_t_Brusselator.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


# Run Koch-Meinhardt's ASDM model with given diffusion constants and correct space-time scalse to find other parameters

In [3]:
layers = [3, 64, 64, 64, 64, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = ASDM(dtype=tf.float64, D_a = 0.0016, D_s = 0.0131)

In [4]:
model = tu.TINN(pinn, pde_loss, alpha = 0.5)

In [5]:
results = model.train(epochs=4,
                      batch_size=512,
                      X=obs_X,
                      Y=obs_Y,
                      print_interval=1,
                      stop_threshold=1e-5,
                      shuffle=True,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0


2022-07-27 09:47:20.449235: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-07-27 09:47:26.518237: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


Training observations acc over epoch: 18.99233
total loss: 59.43080, total regularisd loss (sum of batches): 30428.56874
obs u loss: 24.43181, obs v loss: 13.55288
pde u loss: 1.91459, pde v loss: 19.53152
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a: 10.08218 sigma_s: 9.65446 mu_a: 9.71420 rho_a: 10.53130 
rho_s: 10.52551 kappa_a: 9.45008 
Time taken: 25.58s

Start of epoch 1
Training observations acc over epoch: 17.75725
total loss: 42.83414, total regularisd loss (sum of batches): 21931.08157
obs u loss: 23.24850, obs v loss: 12.26601
pde u loss: 0.13381, pde v loss: 7.18583
lambda obs u: 2.13867, lambda obs v: 0.76335
lambda pde u: 0.50004, lambda pde v: 0.59794
sigma_a: 10.27159 sigma_s: 9.28265 mu_a: 9.28088 rho_a: 11.02895 
rho_s: 10.98063 kappa_a: 8.95726 
Time taken: 15.17s

Start of epoch 2
Training observations acc over epoch: 15.92249
total loss: 38.07537, total regularisd loss (sum of batches): 29599.01827
obs u loss: 21

In [6]:
results.keys()

dict_keys(['training_obs_accuracy', 'loss_total', 'loss_regularisd_total', 'loss_obs_u', 'loss_obs_v', 'loss_pde_u', 'loss_pde_v', 'lambda_obs_u', 'lambda_obs_v', 'lambda_pde_u', 'lambda_pde_v', 'grads_obs_u', 'grads_obs_v', 'grads_pde_u', 'grads_pde_v', 'sigma_a', 'sigma_s', 'mu_a', 'rho_a', 'rho_s', 'kappa_a'])

In [7]:
results['sigma_a']

array([10.08218076, 10.27159382, 10.68180004, 11.16499557])

In [9]:
for name in pde_loss.parameter_names():
    print(name, results[name])

sigma_a [10.08218076 10.27159382 10.68180004 11.16499557]
sigma_s [9.65445654 9.28265265 9.0480151  8.9016571 ]
mu_a [9.71420314 9.28087797 8.57240876 7.87421198]
rho_a [10.53129551 11.02895166 11.71465985 12.33421926]
rho_s [10.52550851 10.98062986 11.24922482 11.41651365]
kappa_a [9.45008129 8.95726171 8.57705994 8.2415736 ]
